<a href="https://colab.research.google.com/github/christianwarmuth/transformer_adapter_bias_evaluation/blob/main/code/adapter_evaluation/adapter_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Jul 25 23:36:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
#!pip install transformers==3.0.2
!pip3 install -r /content/gdrive/MyDrive/master_hpi/NLP_Project/requirements.txt

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-b88p64g4
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-b88p64g4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 748.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.


In [4]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [5]:
import sys
from collections import defaultdict 
from collections import defaultdict 
from transformers import BertForMaskedLM, BertModel, BertTokenizer, BertForSequenceClassification

sys.path.append('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/')
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"

In [6]:
#adapter_list = ["sst"]
#for adapter in adapter_list:
#  print("!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1")

## Sentiment Bert


In [7]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/')
except OSError as e:
        pass

In [8]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sentiment_bert/SentimentBert.pth --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/predictions.json
#!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1

Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sentiment_bert/SentimentBert.pth
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selec

In [9]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/summary.json

intrasentence
	gender
		Count: 765.0
		LM Score: 36.45418751940491
		SS Score: 42.02009600705252
		ICAT Score: 30.636169188489802
	profession
		Count: 2430.0
		LM Score: 45.61401894636235
		SS Score: 45.28042588097158
		ICAT Score: 41.30844408067987
	race
		Count: 2886.0
		LM Score: 49.101373831914486
		SS Score: 70.14426379380126
		ICAT Score: 29.319153289751775
	religion
		Count: 237.0
		LM Score: 44.7816091954023
		SS Score: 50.62068965517241
		ICAT Score: 44.225699564011094
	overall
		Count: 2106.0
		LM Score: 46.01211049519047
		SS Score: 56.40087724103842
		ICAT Score: 40.121753077574276
intersentence
	gender
		Count: 726.0
		LM Score: 85.78311081028474
		SS Score: 58.763293871989525
		ICAT Score: 70.74825862460541
	profession
		Count: 2481.0
		LM Score: 80.69744136512767
		SS Score: 65.2029215734073
		ICAT Score: 56.160703920154276
	race
		Count: 2928.0
		LM Score: 84.90385064737741
		SS Score: 70.48116164220478
		ICAT Score: 50.12526086428642
	religion
		Count: 234.0
		LM Score

## SST Adapter


In [10]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/')
except OSError as e:
        pass

In [11]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst/pytorch_model.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/predictions.json
#!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1

Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst/pytorch_model.bin
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `

In [12]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/summary.json

intrasentence
	gender
		Count: 765.0
		LM Score: 71.29913114260941
		SS Score: 52.07945918815484
		ICAT Score: 68.33385847537029
	profession
		Count: 2430.0
		LM Score: 72.56864556101922
		SS Score: 56.74488262370974
		ICAT Score: 62.77930563160583
	race
		Count: 2886.0
		LM Score: 71.51373905016403
		SS Score: 52.60847925562544
		ICAT Score: 67.78289695407275
	religion
		Count: 237.0
		LM Score: 67.37931034482759
		SS Score: 46.39080459770114
		ICAT Score: 62.51560840269519
	overall
		Count: 2106.0
		LM Score: 71.73016727970962
		SS Score: 53.87618655175265
		ICAT Score: 66.16937708441804
intersentence
	gender
		Count: 726.0
		LM Score: 86.24073189290581
		SS Score: 55.11137212224169
		ICAT Score: 77.42456243692344
	profession
		Count: 2481.0
		LM Score: 83.2999470976297
		SS Score: 60.43945437827529
		ICAT Score: 65.90782714886069
	race
		Count: 2928.0
		LM Score: 85.72979417621613
		SS Score: 59.99013544304608
		ICAT Score: 68.60074906971889
	religion
		Count: 234.0
		LM Score: 85.3